In [121]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px


In [122]:
data = pd.read_csv('../../datas/Store Sales/train.csv')
holidays = pd.read_csv('../../datas/Store Sales/holidays_events.csv')
oil_price = pd.read_csv('../../datas/Store Sales/oil.csv')
transactions = pd.read_csv('../../datas/Store Sales/transactions.csv')


In [123]:
data = data.dropna(ignore_index=True)
holidays = holidays.dropna(ignore_index=True)
oil_price = oil_price.dropna(ignore_index=True)
transactions = transactions.dropna(ignore_index=True)

In [124]:
holidays.head()

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


In [125]:
oil_price.head()

,date,dcoilwtico
0,2013-01-02,93.14
1,2013-01-03,92.97
2,2013-01-04,93.12
3,2013-01-07,93.20
4,2013-01-08,93.21


In [126]:
transactions.head()

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


In [127]:
data.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [128]:
data['date'] = pd.to_datetime(data['date'])

EDA

In [129]:
data = data[data['date'].dt.year == 2017]
data = data[data['date'].dt.month.isin([6, 7, 8])]
data

,id,date,store_nbr,family,sales,onpromotion
2865456,2865456,2017-06-01,1,AUTOMOTIVE,6.000,0
2865457,2865457,2017-06-01,1,BABY CARE,0.000,0
2865458,2865458,2017-06-01,1,BEAUTY,1.000,0
2865459,2865459,2017-06-01,1,BEVERAGES,2250.000,9
2865460,2865460,2017-06-01,1,BOOKS,3.000,0
...,...,...,...,...,...,...
3000883,3000883,2017-08-15,9,POULTRY,438.133,0
3000884,3000884,2017-08-15,9,PREPARED FOODS,154.553,1
3000885,3000885,2017-08-15,9,PRODUCE,2419.729,148
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8


In [139]:
data.value_counts('store_nbr')[:5]

store_nbr
1     2508
41    2508
30    2508
31    2508
32    2508
Name: count, dtype: int64

EDA on single store

In [130]:
# 
data.groupby(['store_nbr'])['sales'].sum().sort_values(ascending=False)[:5]

store_nbr
44    3.439429e+06
45    3.183741e+06
47    2.954475e+06
3     2.814817e+06
49    2.693547e+06
Name: sales, dtype: float64

In [131]:
store_44 = data[data['store_nbr'] == 44]
store_44.head()

,id,date,store_nbr,family,sales,onpromotion
2866710,2866710,2017-06-01,44,AUTOMOTIVE,12.0,0
2866711,2866711,2017-06-01,44,BABY CARE,0.0,0
2866712,2866712,2017-06-01,44,BEAUTY,11.0,0
2866713,2866713,2017-06-01,44,BEVERAGES,8799.0,19
2866714,2866714,2017-06-01,44,BOOKS,0.0,0


In [132]:
store_44.shape

(2508, 6)

In [140]:
store_44_category = store_44.groupby(['date', 'family'])['sales'].sum().reset_index()
pivot_df = store_44_category.pivot(index='date', columns='family', values='sales').fillna(0)
pivot_df = pivot_df.reset_index()
pivot_df

family,date,AUTOMOTIVE,BABY CARE,BEAUTY,BEVERAGES,BOOKS,BREAD/BAKERY,CELEBRATION,CLEANING,DAIRY,...,MAGAZINES,MEATS,PERSONAL CARE,PET SUPPLIES,PLAYERS AND ELECTRONICS,POULTRY,PREPARED FOODS,PRODUCE,SCHOOL AND OFFICE SUPPLIES,SEAFOOD
0,2017-06-01,12.0,0.0,11.0,8799.0,0.0,1535.044,23.0,2281.0,2222.0,...,27.0,777.946,473.0,26.0,33.0,1230.115,387.447,8069.847,8.0,80.406000
1,2017-06-02,14.0,0.0,13.0,12792.0,0.0,1749.278,41.0,2814.0,3225.0,...,24.0,1823.328,846.0,40.0,37.0,2211.133,309.654,9086.198,2.0,115.625000
2,2017-06-03,30.0,0.0,19.0,17039.0,0.0,2279.099,53.0,3720.0,3999.0,...,27.0,1264.210,1370.0,105.0,41.0,1693.300,372.564,12710.305,12.0,106.251000
3,2017-06-04,29.0,0.0,27.0,20384.0,0.0,2646.282,13.0,4131.0,4712.0,...,37.0,1395.614,1550.0,95.0,62.0,2096.785,392.514,13958.719,16.0,120.829000
4,2017-06-05,16.0,0.0,12.0,10439.0,0.0,1695.284,26.0,3030.0,2734.0,...,23.0,853.007,746.0,50.0,24.0,1272.049,408.873,9131.357,4.0,95.707000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,2017-08-11,25.0,0.0,16.0,9517.0,0.0,1772.026,13.0,2106.0,2644.0,...,36.0,1435.457,773.0,51.0,48.0,1489.224,276.162,9311.460,380.0,127.927000
72,2017-08-12,12.0,0.0,19.0,9624.0,0.0,1625.735,23.0,2143.0,2531.0,...,34.0,906.706,595.0,44.0,32.0,1238.589,263.702,9153.445,611.0,87.267000
73,2017-08-13,20.0,0.0,17.0,10491.0,0.0,1843.081,19.0,2300.0,2943.0,...,33.0,829.935,651.0,56.0,41.0,1292.238,281.720,11201.803,424.0,107.092000
74,2017-08-14,10.0,0.0,7.0,7322.0,0.0,1481.496,12.0,1973.0,2211.0,...,27.0,735.229,489.0,36.0,37.0,1028.516,297.046,8735.741,326.0,81.965004


In [141]:
melted_df = pivot_df.melt(id_vars=['date'], value_vars=pivot_df.columns[1:], var_name='family', value_name='sales')
melted_df

,date,family,sales
0,2017-06-01,AUTOMOTIVE,12.000000
1,2017-06-02,AUTOMOTIVE,14.000000
2,2017-06-03,AUTOMOTIVE,30.000000
3,2017-06-04,AUTOMOTIVE,29.000000
4,2017-06-05,AUTOMOTIVE,16.000000
...,...,...,...
2503,2017-08-11,SEAFOOD,127.927000
2504,2017-08-12,SEAFOOD,87.267000
2505,2017-08-13,SEAFOOD,107.092000
2506,2017-08-14,SEAFOOD,81.965004


In [142]:
# Plot using Plotly
fig = px.line(melted_df, x='date', y='sales', color='family', title='Sales Over Time for Store 44')

# Update layout
fig.update_layout(
    xaxis_title='Date',
    yaxis_title='Sales',
    title='Sales Over Time for Store 44',
    legend_title='Family'
)

# Show the plot
fig.show()